In [143]:
import pandas as pd
import numpy as np
import glob
import os

In [144]:
# Create the code to read a csv file 
dtype_spec = {2: str, 3: str, 5: str}

# Load UN + historical dataset
population_df = pd.read_csv('./finalDataset/mergedHistoricalAndProjectionData.csv', dtype=dtype_spec)

In [145]:
population_df.dtypes

SortOrder      float64
LocID          float64
ISO3_code       object
ISO2_code       object
LocTypeID      float64
LocTypeName     object
ParentID       float64
Location        object
VarID          float64
Variant         object
Time             int64
MidPeriod      float64
PopTotal       float64
dtype: object

In [146]:
# CSV files
IHME_ref_path = 'IHME_Projections_Ref.csv'
IHME_SDG_path = 'IHME_Projections_SDG.csv'

# directory to save new file
output_path = './finalDataset/'

IHME_ref_df = pd.read_csv(IHME_ref_path)
IHME_sdg_df = pd.read_csv(IHME_SDG_path)

IHME_ref_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892416 entries, 0 to 892415
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   location_id     892416 non-null  int64  
 1   location_name   892416 non-null  object 
 2   sex_id          892416 non-null  int64  
 3   sex             892416 non-null  object 
 4   age_group_id    892416 non-null  int64  
 5   age_group_name  892416 non-null  object 
 6   year_id         892416 non-null  int64  
 7   measure_id      892416 non-null  int64  
 8   measure_name    892416 non-null  object 
 9   metric_id       892416 non-null  int64  
 10  metric_name     892416 non-null  object 
 11  scenario        892416 non-null  int64  
 12  scenario_name   892416 non-null  object 
 13  val             892416 non-null  float64
 14  upper           892416 non-null  float64
 15  lower           892416 non-null  float64
dtypes: float64(3), int64(7), object(6)
memory usage: 108.9+ 

In [147]:
# Define the column mapping: old name -> new name
column_mapping = {
    # 'Code': 'ISO3_code',
    'year_id': 'Time',
    'val': 'PopTotal',
    'location_name': 'Location',
    'scenario_name': 'Variant'
}

In [148]:
# Rename the columns using the mapping dictionary
IHME_ref_df.rename(columns=column_mapping, inplace=True)
IHME_sdg_df.rename(columns=column_mapping, inplace=True)

In [149]:
# Define the list of Year values to keep
years_to_keep = [2050, 2100]

# Filter dataframe to only rows where 'Year' is in years_to_keep
IHME_ref_df = IHME_ref_df[IHME_ref_df['Time'].isin(years_to_keep)]
IHME_sdg_df = IHME_sdg_df[IHME_sdg_df['Time'].isin(years_to_keep)]

In [150]:
# Define the list of Age values to keep
ages_to_keep = ["All Ages"]

# Filter dataframe to only rows where 'Year' is in years_to_keep
IHME_ref_df = IHME_ref_df[IHME_ref_df['age_group_name'].isin(ages_to_keep)]
IHME_sdg_df = IHME_sdg_df[IHME_sdg_df['age_group_name'].isin(ages_to_keep)]

In [151]:
# Add variant name 
IHME_ref_df['Variant'] = 'IHME reference pace'
IHME_ref_df['LocTypeID'] = 4.0

IHME_sdg_df['Variant'] = 'IHME sustainable development goals (SDG) pace'
IHME_sdg_df['LocTypeID'] = 4.0

In [152]:
IHME_sdg_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 896 entries, 706528 to 743679
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   location_id     896 non-null    int64  
 1   Location        896 non-null    object 
 2   sex_id          896 non-null    int64  
 3   sex             896 non-null    object 
 4   age_group_id    896 non-null    int64  
 5   age_group_name  896 non-null    object 
 6   Time            896 non-null    int64  
 7   measure_id      896 non-null    int64  
 8   measure_name    896 non-null    object 
 9   metric_id       896 non-null    int64  
 10  metric_name     896 non-null    object 
 11  scenario        896 non-null    int64  
 12  Variant         896 non-null    object 
 13  PopTotal        896 non-null    float64
 14  upper           896 non-null    float64
 15  lower           896 non-null    float64
 16  LocTypeID       896 non-null    float64
dtypes: float64(4), int64(7), object(

In [153]:
# Drop unnecessary columns
IHME_ref_df.drop(columns=['location_id', 'sex_id', 'age_group_id', 'age_group_name', 'measure_id', 
                         'measure_name', 'metric_id', 'metric_name', 'scenario', 'upper', 'lower'], inplace=True)

IHME_sdg_df.drop(columns=['location_id', 'sex_id', 'age_group_id', 'age_group_name', 'measure_id', 
                         'measure_name', 'metric_id', 'metric_name', 'scenario', 'upper', 'lower'], inplace=True)

In [154]:
# Cast PopTotal to float
IHME_ref_df['PopTotal'] = IHME_ref_df['PopTotal'].astype(float)
IHME_sdg_df['PopTotal'] = IHME_sdg_df['PopTotal'].astype(float)

In [155]:
# Format in thousands
IHME_ref_df['PopTotal'] = IHME_ref_df['PopTotal'] / 1000
IHME_sdg_df['PopTotal'] = IHME_sdg_df['PopTotal'] / 1000

In [156]:
IHME_ref_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 896 entries, 706528 to 743679
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Location   896 non-null    object 
 1   sex        896 non-null    object 
 2   Time       896 non-null    int64  
 3   Variant    896 non-null    object 
 4   PopTotal   896 non-null    float64
 5   LocTypeID  896 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 49.0+ KB


In [157]:
IHME_ref_df.head()

Location   sex  Time  \
706528                                            Global  Male  2050   
706578                                            Global  Male  2100   
706611  Central Europe, Eastern Europe, and Central Asia  Male  2050   
706661  Central Europe, Eastern Europe, and Central Asia  Male  2100   
706694                                      Central Asia  Male  2050   

                    Variant      PopTotal  LocTypeID  
706528  IHME reference pace  4.776133e+06        4.0  
706578  IHME reference pace  4.401326e+06        4.0  
706611  IHME reference pace  1.916529e+05        4.0  
706661  IHME reference pace  1.594659e+05        4.0  
706694  IHME reference pace  6.013316e+04        4.0

In [184]:
# Suppose your DataFrame has columns: ['Year', 'Variant', 'Location', 'sex', 'Population']
# Keep everything the same except sex, summing over 'Male' and 'Female'

IHME_ref_df_total = (
    IHME_ref_df
    .groupby(['Location', 'Time', 'Variant', 'LocTypeID'], as_index=False) # list all the columns EXCEPT 'sex'
    ['PopTotal'].sum() # sum the population across the grouped rows
)

IHME_sdg_df_total = (
    IHME_sdg_df
    .groupby(['Location', 'Time', 'Variant', 'LocTypeID'], as_index=False) # list all the columns EXCEPT 'sex'
    ['PopTotal'].sum() # sum the population across the grouped rows
)

IHME_sdg_df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 444 entries, 0 to 443
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Location   444 non-null    object 
 1   Time       444 non-null    int64  
 2   Variant    444 non-null    object 
 3   LocTypeID  444 non-null    float64
 4   PopTotal   444 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 17.5+ KB


In [185]:
import pycountry

# Handle iso code issues (the dataset doesn't have iso codes)
def get_country_code(country_name):
    try:
        # Attempt to look up the country by name
        return pycountry.countries.search_fuzzy(country_name)[0].alpha_3
    except:
        # If not found, return None or some placeholder
        return None

IHME_ref_df_total["ISO3_code"] = IHME_ref_df_total["Location"].apply(get_country_code)
IHME_sdg_df_total["ISO3_code"] = IHME_sdg_df_total["Location"].apply(get_country_code)

In [186]:
IHME_sdg_df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 444 entries, 0 to 443
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Location   444 non-null    object 
 1   Time       444 non-null    int64  
 2   Variant    444 non-null    object 
 3   LocTypeID  444 non-null    float64
 4   PopTotal   444 non-null    float64
 5   ISO3_code  374 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 20.9+ KB


In [193]:
df_south_korea = IHME_ref_df_total[IHME_sdg_df_total["Location"].str.contains("Korea", case=False, regex=True)]
df_south_korea

Location  Time              Variant  \
110  Democratic People's Republic of Korea  2050  IHME reference pace   
111  Democratic People's Republic of Korea  2100  IHME reference pace   
320                      Republic of Korea  2050  IHME reference pace   
321                      Republic of Korea  2100  IHME reference pace   

     LocTypeID      PopTotal ISO3_code  
110        4.0  23538.251610       PRK  
111        4.0  12975.869826       PRK  
320        4.0  49976.482150       KOR  
321        4.0  26777.352390       KOR

In [188]:
erroneous_ISO_fix = {
    'Republic of Korea': 'KOR',
    'Niger': 'NER'
}

# For each row, if the "Country" is in manual_fixes,
# override the ISO3 with the known correct code
IHME_ref_df_total["ISO3_code"] = IHME_ref_df_total.apply(
    lambda row: erroneous_ISO_fix[row["Location"]]
    if row["Location"] in erroneous_ISO_fix else row["ISO3_code"],
    axis=1
)

IHME_sdg_df_total["ISO3_code"] = IHME_sdg_df_total.apply(
    lambda row: erroneous_ISO_fix[row["Location"]]
    if row["Location"] in erroneous_ISO_fix else row["ISO3_code"],
    axis=1
)

In [194]:
# unmatched = IHME_ref_df_total[IHME_ref_df_total['ISO_Code'].isna()]
# unmatched.head(50)

# 'Democratic Republic of the Congo' = 'Dem. Rep. Congo'
# 'Bolivia (Plurinational State of)' = 'Bolivia'
# 'Micronesia (Federated States of)' = 'Micronesia'
# 'Iran (Islamic Republic of)' = 'Iran'
# 'Turkey' = 'Türkiye'
# 'Taiwan (Province of China)' = 'Taiwan'
# 'Venezuela (Bolivarian Republic of)' = 'Venezuela'

# Missing ISO code mapping
country_to_ISO_mapping = {
    'Democratic Republic of the Congo': 'COD',
    'Bolivia (Plurinational State of)': 'BOL',
    'Micronesia (Federated States of)': 'FSM',
    'Iran (Islamic Republic of)': 'IRN',
    'Turkey': 'TUR',
    'Taiwan (Province of China)': 'TWN',
    'Venezuela (Bolivarian Republic of)': 'VEN'
}

# Get a Series of mapped values
mapped_series = IHME_ref_df_total["Location"].map(country_to_ISO_mapping)

# Fill in only where df["ISO3_Code"] is currently NaN
IHME_ref_df_total["ISO3_code"] = IHME_ref_df_total["ISO3_code"].fillna(mapped_series)
IHME_sdg_df_total["ISO3_code"] = IHME_sdg_df_total["ISO3_code"].fillna(mapped_series)

In [195]:
IHME_ref_df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 444 entries, 0 to 443
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Location   444 non-null    object 
 1   Time       444 non-null    int64  
 2   Variant    444 non-null    object 
 3   LocTypeID  444 non-null    float64
 4   PopTotal   444 non-null    float64
 5   ISO3_code  388 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 20.9+ KB


In [196]:
# Create a mapping from ISO3_code to the standardized country name (Location)
mapping = population_df[['ISO3_code', 'Location']].drop_duplicates().set_index('ISO3_code')['Location'].to_dict()

# Map the standardized country names from the Big Dataset into the Little Dataset based on ISO3_code
IHME_ref_df_total['Location'] = IHME_ref_df_total['ISO3_code'].map(mapping)
IHME_sdg_df_total['Location'] = IHME_sdg_df_total['ISO3_code'].map(mapping)

In [197]:
IHME_sdg_df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 444 entries, 0 to 443
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Location   444 non-null    object 
 1   Time       444 non-null    int64  
 2   Variant    444 non-null    object 
 3   LocTypeID  444 non-null    float64
 4   PopTotal   444 non-null    float64
 5   ISO3_code  388 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 20.9+ KB


In [198]:
# Merge
population_df = pd.concat([population_df, IHME_ref_df_total], ignore_index=True, sort=False)
population_df = pd.concat([population_df, IHME_sdg_df_total], ignore_index=True, sort=False)

# Save final file
new_file = os.path.join(output_path, "mergedHistoricalAndProjectionData2.csv")
    
# Save the DataFrame to a new CSV file
population_df.to_csv(new_file, index=False)

In [199]:
population_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384937 entries, 0 to 384936
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   SortOrder    221490 non-null  float64
 1   LocID        382950 non-null  float64
 2   ISO3_code    165405 non-null  object 
 3   ISO2_code    162840 non-null  object 
 4   LocTypeID    223477 non-null  float64
 5   LocTypeName  221490 non-null  object 
 6   ParentID     221490 non-null  float64
 7   Location     384937 non-null  object 
 8   VarID        382950 non-null  float64
 9   Variant      384937 non-null  object 
 10  Time         384937 non-null  int64  
 11  MidPeriod    382950 non-null  float64
 12  PopTotal     384937 non-null  float64
dtypes: float64(7), int64(1), object(5)
memory usage: 38.2+ MB


In [200]:
dups = population_df[population_df.duplicated(subset=['ISO3_code', 'Time', 'Variant'])]
dups.info()

<class 'pandas.core.frame.DataFrame'>
Index: 218838 entries, 690 to 384930
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   SortOrder    57960 non-null   float64
 1   LocID        218730 non-null  float64
 2   ISO3_code    0 non-null       object 
 3   ISO2_code    0 non-null       object 
 4   LocTypeID    58068 non-null   float64
 5   LocTypeName  57960 non-null   object 
 6   ParentID     57960 non-null   float64
 7   Location     218838 non-null  object 
 8   VarID        218730 non-null  float64
 9   Variant      218838 non-null  object 
 10  Time         218838 non-null  int64  
 11  MidPeriod    218730 non-null  float64
 12  PopTotal     218838 non-null  float64
dtypes: float64(7), int64(1), object(5)
memory usage: 23.4+ MB


In [201]:
dups_not_null = dups[dups["ISO3_code"].notnull()]
dups_not_null

Empty DataFrame
Columns: [SortOrder, LocID, ISO3_code, ISO2_code, LocTypeID, LocTypeName, ParentID, Location, VarID, Variant, Time, MidPeriod, PopTotal]
Index: []